In [1]:
import pandas as pd

output_correct = pd.DataFrame()

output_error = pd.DataFrame()

In [2]:
input_ = pd.read_excel('./2.category.xlsx',
                       header=0,
                       dtype=str).fillna('')

length = len(input_)

print('总数量：' + str(length))
input_['Category'].tolist()

总数量：2472


['Body & Lamp Assembly',
 'Interior',
 'Body & Lamp Assembly',
 'Engine',
 'Fuel & Air',
 'Garage Equipment',
 'Interior',
 'Steering',
 'Interior',
 'Ignition',
 'Belt Drive',
 'Body & Lamp Assembly',
 'Brake & Wheel Hub',
 'Cooling System',
 'Drivetrain',
 'Electrical',
 'Electrical-Bulb & Socket',
 'Electrical-Connector',
 'Electrical-Switch & Relay',
 'Engine',
 'Exhaust & Emission',
 'Fuel & Air',
 'Garage Equipment',
 'Hardware',
 'Heat & Air Conditioning',
 'Hoses/Lines & Clamps',
 'Ignition',
 'Interior',
 'Steering',
 'Suspension',
 'Transmission-Automatic',
 'Transmission-Manual',
 'Wheel',
 'Wiper & Washer',
 'Body & Lamp Assembly',
 'Electrical-Connector',
 'Electrical-Switch & Relay',
 'Garage Equipment',
 'Heat & Air Conditioning',
 'Interior',
 'Wiper & Washer',
 'Body & Lamp Assembly',
 'Interior',
 'Ignition',
 'Brake & Wheel Hub',
 'Fuel & Air',
 'Cooling System',
 'Engine',
 'Heat & Air Conditioning',
 'Wiper & Washer',
 'Brake & Wheel Hub',
 'Electrical',
 'Engine',

In [3]:
crawler_status = 'error'

print(crawler_status)

error


In [4]:
import requests
import json

In [5]:
i = 0

payload = {'func': 'partsearchgetparttypessdd',
           'mfr': input_.loc[i, 'Manufacturer'],
           'pg': input_.loc[i, 'Category'],
           'api_json_request': '1'}

resp = requests.post('https://www.rockauto.com/catalog/catalogapi.php',
                     data=payload).text

resp = json.loads(resp)

resp

{'html_replace_sections': {'ddrepl[parttype_][container]': '<span id="ddrepl[parttype_][container]" class="ra-vertical-dropdown-replacement-c"><select id="parttype_" name="partsearch[parttype][]" onchange = "" tabindex="0" autocomplete = "none"><option value=""  selected >All</option><option value="1263"  >Cargo Bag</option><option value="1000410"  >Cargo Carrier</option><option value="1104"  >Roof Rack</option><option value="1152"  >Sunroof Wind Deflector</option><option value="1427"  >Tailgate</option></select></span>'},
 'collected_javascript': '/**UQ:1F113E5D**/\nwindow.top.parent.window["_nck"] = "8LVLUxTaILBT+tLxkg8+3Ag7\\/0bbOhvn7P3rkFbL80\\/oQB6RvzNfa5dcHvATqXAMxw5whzaDEdxk+27zeij67BjKru6WqTdNqG6tiEsD1EZhbTv0XForkbYp+nLaIKxDXfb4DjpqS2ZAZtComwNL2PHGKN3aeD0dgq3AkPxkRtA=";\n'}

In [6]:
from bs4 import BeautifulSoup

In [7]:
html = resp['html_replace_sections']['ddrepl[parttype_][container]']

soup = BeautifulSoup(html, 'lxml')

with open('./type.html', 'w', encoding='UTF-8') as file:
    file.write(str(soup))

print(soup.prettify())

<html>
 <body>
  <span class="ra-vertical-dropdown-replacement-c" id="ddrepl[parttype_][container]">
   <select autocomplete="none" id="parttype_" name="partsearch[parttype][]" onchange="" tabindex="0">
    <option selected="" value="">
     All
    </option>
    <option value="1263">
     Cargo Bag
    </option>
    <option value="1000410">
     Cargo Carrier
    </option>
    <option value="1104">
     Roof Rack
    </option>
    <option value="1152">
     Sunroof Wind Deflector
    </option>
    <option value="1427">
     Tailgate
    </option>
   </select>
  </span>
 </body>
</html>



In [8]:
from lxml import etree

In [ ]:
html = etree.HTML(str(soup))

security_code = html.xpath('//div[@class="center"]/div')

security_code

In [ ]:
len(security_code) != 0 and 'security code' in security_code[0].text

In [9]:
list_type = [type_.strip() for type_ in html.xpath('//select[@id="parttype_"]/option/text()')[1:]]

list_type

['Cargo Bag',
 'Cargo Carrier',
 'Roof Rack',
 'Sunroof Wind Deflector',
 'Tailgate']

In [10]:
list_type_code = [type_code.strip() for type_code in html.xpath('//select[@id="parttype_"]/option/@value')[1:]]

list_type_code

['1263', '1000410', '1104', '1152', '1427']

In [ ]:
df_temp = pd.DataFrame({'Tab_Label': input_.loc[i, 'Tab_Label'],
                        'No.': int(input_.loc[i, 'No.']),
                        'Manufacturer': input_.loc[i, 'Manufacturer'],
                        'Category': input_.loc[i, 'Category'],
                        'Type': list_type,
                        'Type_Code': list_type_code,
                        'Url': ['https://www.rockauto.com/en/parts/'+input_.loc[i, 'Manufacturer'].replaceace('+', '%2B').replace(',', '%2C')+','+type_.replace(',', '%2C')+','+type_code for type_, type_code in zip(list_type, list_type_code)]})

output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

output_correct

In [11]:
crawler_status = 'ok'

print(crawler_status)

ok


In [12]:
if crawler_status != 'ok':
    df_temp = pd.DataFrame([{'Tab_Label': input_.loc[i, 'Tab_Label'],
                             'No.': int(input_.loc[i, 'No.']),
                             'Manufacturer': input_.loc[i, 'Manufacturer'],
                             'Category': input_.loc[i, 'Category']}])
    
    output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')
    
output_error

5 0
